# Project

# Notebook Overview: SVD


This particular notebook demonstrates an application of SVD on a truncated dataset. 

The data looks at successfully landed stats for the first instance of a B_fighter. 

-Ultimately, this means that we have to eventually modify the data in that we average of that fighter's record. For example, if Khabib Nurmagomedov, over three matches, had 5,8,3 takedowns, then possibly we can modify that record into 
the average over those instances (meaning: Khabib Nurmagomedov Takedown: 5.3 as opposed to Khabib 3, Khabib 5, Khabib 3). Or double check the cvs zip to see if maybe another dataset offered is more beneficial. 

-Does not cluster by weight class. Currently, weight class is represented by a string, which I have to go through and change to be the numeric value

-Does not cluster by gender. Need to add a bool value row which would indicate if the fighter is male or female. 
This can be filtered by using the weight_class's string, which says "Women's Bantamweight" vs "Bantamweight". 

-TODO: 
In the fighter_indeces, need to get the index that corresponds with the fighter. 
ex: (def fighter_index (fighter) =..... , fighter_index("Amanda Nunes") = 80) 

Need to produces the recommendations list for that fighter using fighter_index 
ex: (def getRecommendationsfor("Amanda Nunes"):.....", getRecommendationsfor("Amanda Nunes") = "Valentina Schevchenko"). 

Note: Schevchenko's just a guess, but more than likely, they would be similar fighters. 



Application starts at In[48]. 

## Data Cleaning and Imputation

We begin by loading the dataset, and showing a few of the matches to prove the data has been loaded.

In [1]:
import pandas as pd
df = pd.read_csv("data.csv")
df.head(5)

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Henry Cejudo,Marlon Moraes,Marc Goddard,2019-06-08,"Chicago, Illinois, USA",Red,True,Bantamweight,5,0.0,...,2.0,0.0,0.0,8.0,Orthodox,162.56,162.56,135.0,31.0,32.0
1,Valentina Shevchenko,Jessica Eye,Robert Madrigal,2019-06-08,"Chicago, Illinois, USA",Red,True,Women's Flyweight,5,0.0,...,0.0,2.0,0.0,5.0,Southpaw,165.10,167.64,125.0,32.0,31.0
2,Tony Ferguson,Donald Cerrone,Dan Miragliotta,2019-06-08,"Chicago, Illinois, USA",Red,False,Lightweight,3,0.0,...,3.0,6.0,1.0,14.0,Orthodox,180.34,193.04,155.0,36.0,35.0
3,Jimmie Rivera,Petr Yan,Kevin MacDonald,2019-06-08,"Chicago, Illinois, USA",Blue,False,Bantamweight,3,0.0,...,1.0,0.0,0.0,6.0,Orthodox,162.56,172.72,135.0,26.0,29.0
4,Tai Tuivasa,Blagoy Ivanov,Dan Miragliotta,2019-06-08,"Chicago, Illinois, USA",Blue,False,Heavyweight,3,0.0,...,2.0,0.0,0.0,3.0,Southpaw,187.96,190.50,264.0,32.0,26.0


### Missing Stances and Referee

Some of the matches feature fighters with missing stance information or missing referee information. Since this is a relatively small number of matches out of the whole dataset, we will drop these matches from our consideration.

In [2]:
# Show number of matches in dataset before removing matches with missing stance information
print('Number of matches prior to filtering: ' + str(len(df)))

# Remove matches with missing stance information
filter1 = df[df['B_Stance'].notnull()]
filter2 = filter1[filter1['R_Stance'].notnull()]
filter3 = filter2[filter2['Referee'].notnull()]
df = filter3
print('Number of matchs after filtering: ' + str(len(filter3)))

Number of matches prior to filtering: 5144
Number of matchs after filtering: 4865


### Missing Numerical Data

Many of the rows contain missing numerical data in certain columns. We will fill in these columns with the median for that column.

First we will find columns with missing numerical data to demonstrate that our imputation is successful.

In [3]:
df.columns[df.isnull().any()]

Index(['B_avg_BODY_att', 'B_avg_BODY_landed', 'B_avg_CLINCH_att',
       'B_avg_CLINCH_landed', 'B_avg_DISTANCE_att', 'B_avg_DISTANCE_landed',
       'B_avg_GROUND_att', 'B_avg_GROUND_landed', 'B_avg_HEAD_att',
       'B_avg_HEAD_landed',
       ...
       'R_avg_opp_SUB_ATT', 'R_avg_opp_TD_att', 'R_avg_opp_TD_landed',
       'R_avg_opp_TD_pct', 'R_avg_opp_TOTAL_STR_att',
       'R_avg_opp_TOTAL_STR_landed', 'R_total_time_fought(seconds)',
       'R_Reach_cms', 'B_age', 'R_age'],
      dtype='object', length=104)

We select R_age for our demonstration. Now we find a few rows that have a missing R_age.

In [4]:
people = df[df['R_age'].isnull()].head(5)
people

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
4171,Per Eklund,Samy Schiavo,Leon Roberts,2008-10-18,"Birmingham, England, United Kingdom",Red,False,Lightweight,3,1.0,...,0.0,0.0,0.0,0.0,Orthodox,177.80,182.88,155.0,32.0,NaN
4376,Jess Liaudin,Anthony Torres,Mario Yamasaki,2007-09-08,"London, England, United Kingdom",Red,False,Welterweight,3,0.0,...,0.0,1.0,0.0,1.0,Orthodox,175.26,182.88,170.0,29.0,NaN
4438,Jess Liaudin,Dennis Siver,Steve Mazzagatti,2007-04-21,"Manchester, England, United Kingdom",Red,False,Welterweight,3,0.0,...,0.0,0.0,0.0,0.0,Orthodox,175.26,182.88,170.0,28.0,NaN
4767,Keith Rockel,Chris Liguori,John McCarthy,2003-11-21,"Uncasville, Connecticut, USA",Red,False,Middleweight,3,0.0,...,0.0,0.0,0.0,0.0,Orthodox,182.88,NaN,185.0,NaN,NaN
4908,Ben Earwood,Chris Lytle,Mario Yamasaki,2000-11-17,"Atlantic City, New Jersey, USA",Red,False,Welterweight,2,0.0,...,0.0,0.0,0.0,0.0,Orthodox,172.72,NaN,170.0,26.0,NaN


Now we find the median for that column.

In [5]:
df['R_age'].median()

29.0

Now we apply fillna to the entire dataset, and show that the column is filled in with the median.

In [6]:
df = df.fillna(df.median())
df.loc[people.index.tolist()]

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
4171,Per Eklund,Samy Schiavo,Leon Roberts,2008-10-18,"Birmingham, England, United Kingdom",Red,False,Lightweight,3,1.0,...,0.0,0.0,0.0,0.0,Orthodox,177.80,182.88,155.0,32.0,29.0
4376,Jess Liaudin,Anthony Torres,Mario Yamasaki,2007-09-08,"London, England, United Kingdom",Red,False,Welterweight,3,0.0,...,0.0,1.0,0.0,1.0,Orthodox,175.26,182.88,170.0,29.0,29.0
4438,Jess Liaudin,Dennis Siver,Steve Mazzagatti,2007-04-21,"Manchester, England, United Kingdom",Red,False,Welterweight,3,0.0,...,0.0,0.0,0.0,0.0,Orthodox,175.26,182.88,170.0,28.0,29.0
4767,Keith Rockel,Chris Liguori,John McCarthy,2003-11-21,"Uncasville, Connecticut, USA",Red,False,Middleweight,3,0.0,...,0.0,0.0,0.0,0.0,Orthodox,182.88,185.42,185.0,29.0,29.0
4908,Ben Earwood,Chris Lytle,Mario Yamasaki,2000-11-17,"Atlantic City, New Jersey, USA",Red,False,Welterweight,2,0.0,...,0.0,0.0,0.0,0.0,Orthodox,172.72,185.42,170.0,26.0,29.0


### Dimensionality Reduction-PCA ### 

We apply dimensionality reduction to obtain an ordered list of components that account for the largest variance in the data set in order to ultimately group similar fighters based on their fighting styles



Firstly, we drop the non numeric columns

# SVD and kNN

In [7]:
#dealing with only the B_fighter and successful attacks landed. Much smaller dataset. 
#reasoning: successful attacks tend are more significatn 
#attacks that were attempted showcase "volume" of that fight which vary match by match
#whereas successfully landed shots showcase a fighter's accuracy/overall control of the match

view = df[["B_fighter", "weight_class", "R_current_lose_streak", "B_current_win_streak", "B_draw", "B_avg_BODY_landed",
           "B_avg_CLINCH_landed", "B_avg_DISTANCE_landed", "B_avg_GROUND_landed", "B_avg_HEAD_landed", "B_avg_KD", 
           "B_avg_LEG_landed", "B_avg_PASS", "B_avg_REV", "B_avg_SIG_STR_landed", "B_avg_SIG_STR_pct", "B_avg_SUB_ATT", 
           "B_avg_TD_landed", "B_avg_TD_pct", "B_avg_TOTAL_STR_landed", "B_longest_win_streak", "B_losses", 
           "B_avg_opp_BODY_landed", "B_avg_opp_CLINCH_landed", "B_avg_opp_DISTANCE_landed", "B_avg_opp_GROUND_landed", 
           "B_avg_opp_HEAD_landed", "B_avg_opp_KD", "B_avg_opp_LEG_landed", "B_avg_opp_PASS", "B_avg_opp_REV", 
           "B_avg_opp_SIG_STR_landed", "B_avg_opp_SIG_STR_pct", "B_avg_opp_TD_landed", "B_avg_opp_TD_pct",
           "B_avg_opp_TOTAL_STR_landed", "B_total_title_bouts", "B_win_by_Decision_Majority", "B_win_by_Decision_Split", 
           "B_win_by_Decision_Unanimous", "B_win_by_KO/TKO", "B_win_by_Submission", "B_win_by_TKO_Doctor_Stoppage", 
           "B_wins"]]



#dropping duplicates
newView = view.drop_duplicates(subset = "B_fighter", keep = "first")

newView.head()

,B_fighter,weight_class,R_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_landed,B_avg_CLINCH_landed,B_avg_DISTANCE_landed,B_avg_GROUND_landed,B_avg_HEAD_landed,...,B_avg_opp_TD_pct,B_avg_opp_TOTAL_STR_landed,B_total_title_bouts,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_wins
0,Marlon Moraes,Bantamweight,0.0,4.0,0.0,6.000000,0.000000,20.600000,2.000000,11.200000,...,0.100000,19.200000,0.0,0.0,1.0,0.0,2.0,1.0,0.0,4.0
1,Jessica Eye,Women's Flyweight,0.0,3.0,0.0,9.100000,7.300000,42.100000,1.900000,32.000000,...,0.231000,75.400000,0.0,0.0,2.0,1.0,0.0,0.0,1.0,4.0
2,Donald Cerrone,Lightweight,0.0,3.0,0.0,11.322581,4.387097,38.580645,3.806452,23.258065,...,0.063548,49.774194,1.0,0.0,0.0,7.0,10.0,6.0,0.0,23.0
3,Petr Yan,Bantamweight,1.0,4.0,0.0,14.000000,11.000000,48.750000,10.500000,53.750000,...,0.097500,34.250000,0.0,0.0,0.0,2.0,2.0,0.0,0.0,4.0
4,Blagoy Ivanov,Heavyweight,1.0,1.0,0.0,14.500000,2.000000,59.500000,0.000000,45.000000,...,0.000000,90.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [8]:
#changing the weight class from string to numeric: 
#incomplete because identified as "women's" and "mens"
newView["weight_class"].unique()

# for i in newView["weight_class"]:
#     if i == "Bantamweight": 
#         df.loc[df['weight_class'] == i] = 135
    
# display(newView["weight_class"])


#as of right now, leaving as is. 


array(['Bantamweight', "Women's Flyweight", 'Lightweight', 'Heavyweight',
       "Women's Strawweight", 'Featherweight', 'Middleweight',
       'Light Heavyweight', 'Welterweight', "Women's Bantamweight",
       "Women's Featherweight", 'Flyweight', 'Catch Weight',
       'Open Weight'], dtype=object)

In [14]:
newView[['B_fighter', "B_wins"]].loc[newView['B_fighter'] == "Khabib Nurmagomedov"]


,B_fighter,B_wins
2478,Khabib Nurmagomedov,5.0


In [9]:
from sklearn.neighbors import NearestNeighbors
#later after reformatting weight_class to have the numerical numbers
#we will add it in. for now, drop since string. 
stats = newView.drop(columns = ["B_fighter", "R_current_lose_streak", "weight_class"])

In [10]:
stats.head()

,B_current_win_streak,B_draw,B_avg_BODY_landed,B_avg_CLINCH_landed,B_avg_DISTANCE_landed,B_avg_GROUND_landed,B_avg_HEAD_landed,B_avg_KD,B_avg_LEG_landed,B_avg_PASS,...,B_avg_opp_TD_pct,B_avg_opp_TOTAL_STR_landed,B_total_title_bouts,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_wins
0,4.0,0.0,6.000000,0.000000,20.600000,2.000000,11.200000,0.800000,5.400000,0.400000,...,0.100000,19.200000,0.0,0.0,1.0,0.0,2.0,1.0,0.0,4.0
1,3.0,0.0,9.100000,7.300000,42.100000,1.900000,32.000000,0.000000,10.200000,0.800000,...,0.231000,75.400000,0.0,0.0,2.0,1.0,0.0,0.0,1.0,4.0
2,3.0,0.0,11.322581,4.387097,38.580645,3.806452,23.258065,0.645161,12.193548,0.935484,...,0.063548,49.774194,1.0,0.0,0.0,7.0,10.0,6.0,0.0,23.0
3,4.0,0.0,14.000000,11.000000,48.750000,10.500000,53.750000,0.500000,2.500000,0.500000,...,0.097500,34.250000,0.0,0.0,0.0,2.0,2.0,0.0,0.0,4.0
4,1.0,0.0,14.500000,2.000000,59.500000,0.000000,45.000000,0.000000,2.000000,0.000000,...,0.000000,90.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [11]:
stats = stats.transpose()

In [12]:
stats.shape

(41, 1662)

In [13]:
x = stats.values.T

In [14]:
x.shape

(1662, 41)

In [15]:
import sklearn 

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components = 12, random_state = 17)

matrix = svd.fit_transform(x)

matrix.shape

(1662, 12)

# Visualizations starting about here

In [20]:

#get the index value for Amanda Nunes
#find its value <1 and >.9

In [9]:
newView

,B_fighter,weight_class,R_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_landed,B_avg_CLINCH_landed,B_avg_DISTANCE_landed,B_avg_GROUND_landed,B_avg_HEAD_landed,...,B_avg_opp_TD_pct,B_avg_opp_TOTAL_STR_landed,B_total_title_bouts,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_wins
0,Marlon Moraes,Bantamweight,0.0,4.0,0.0,6.000000,0.000000,20.600000,2.000000,11.200000,...,0.100000,19.200000,0.0,0.0,1.0,0.0,2.0,1.0,0.0,4.0
1,Jessica Eye,Women's Flyweight,0.0,3.0,0.0,9.100000,7.300000,42.100000,1.900000,32.000000,...,0.231000,75.400000,0.0,0.0,2.0,1.0,0.0,0.0,1.0,4.0
2,Donald Cerrone,Lightweight,0.0,3.0,0.0,11.322581,4.387097,38.580645,3.806452,23.258065,...,0.063548,49.774194,1.0,0.0,0.0,7.0,10.0,6.0,0.0,23.0
3,Petr Yan,Bantamweight,1.0,4.0,0.0,14.000000,11.000000,48.750000,10.500000,53.750000,...,0.097500,34.250000,0.0,0.0,0.0,2.0,2.0,0.0,0.0,4.0
4,Blagoy Ivanov,Heavyweight,1.0,1.0,0.0,14.500000,2.000000,59.500000,0.000000,45.000000,...,0.000000,90.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5128,Minoki Ichihara,Open Weight,0.0,0.0,0.0,4.823529,4.200000,15.000000,4.333333,17.576923,...,0.237143,43.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5136,Gerard Gordeau,Catch Weight,0.0,2.0,0.0,0.500000,0.000000,3.000000,4.000000,5.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0
5139,Kevin Rosier,Open Weight,0.0,1.0,0.0,3.000000,4.000000,4.000000,7.000000,12.000000,...,0.000000,13.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
5141,Art Jimmerson,Open Weight,0.0,0.0,0.0,4.823529,4.200000,15.000000,4.333333,17.576923,...,0.237143,43.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
filtered_df_female_new  = newView.loc[(newView['weight_class'] == "Women's Bantamweight") | (newView['weight_class'] == "Women's Featherweight") | (newView['weight_class'] == "Women's Featherweight") | (newView['weight_class'] == "Women's Flyweight") | (newView['weight_class'] == "Women's Strawweight")]


In [13]:
filtered_df_male_new  = newView.loc[(newView['weight_class'] == "Bantamweight") | (newView['weight_class'] == "Catch Weight") | (newView['weight_class'] == "Featherweight") | (newView['weight_class'] == "Flyweight") | (newView['weight_class'] == "Heavyweight") | (newView['weight_class'] == "Light Heavyweight") | (newView['weight_class'] == "Lightweight") | (newView['weight_class'] == "Middleweight") | (newView['weight_class'] == "Open Weight") | (newView['weight_class'] == "Welterweight") ]

In [14]:
#female 1, male 2
filtered_df_female_new["Gender"]=1
filtered_df_male_new["Gender"]=2

<ipython-input-14-6a76d21b4423>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_female_new["Gender"]=1
<ipython-input-14-6a76d21b4423>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_male_new["Gender"]=2


In [15]:
frames_new=[filtered_df_male_new,filtered_df_female_new]

In [16]:
combine_male_female_df_new=pd.concat(frames_new)

In [17]:
combine_male_female_df_new

,B_fighter,weight_class,R_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_landed,B_avg_CLINCH_landed,B_avg_DISTANCE_landed,B_avg_GROUND_landed,B_avg_HEAD_landed,...,B_avg_opp_TOTAL_STR_landed,B_total_title_bouts,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_wins,Gender
0,Marlon Moraes,Bantamweight,0.0,4.0,0.0,6.000000,0.000000,20.600000,2.000000,11.200000,...,19.200000,0.0,0.0,1.0,0.0,2.0,1.0,0.0,4.0,2
2,Donald Cerrone,Lightweight,0.0,3.0,0.0,11.322581,4.387097,38.580645,3.806452,23.258065,...,49.774194,1.0,0.0,0.0,7.0,10.0,6.0,0.0,23.0,2
3,Petr Yan,Bantamweight,1.0,4.0,0.0,14.000000,11.000000,48.750000,10.500000,53.750000,...,34.250000,0.0,0.0,0.0,2.0,2.0,0.0,0.0,4.0,2
4,Blagoy Ivanov,Heavyweight,1.0,1.0,0.0,14.500000,2.000000,59.500000,0.000000,45.000000,...,90.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2
6,Pedro Munhoz,Bantamweight,0.0,3.0,0.0,7.416667,3.416667,34.666667,3.416667,24.000000,...,46.500000,0.0,0.0,0.0,2.0,3.0,3.0,0.0,8.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2073,Sara McMann,Women's Bantamweight,0.0,1.0,0.0,4.000000,3.666667,9.333333,6.666667,12.666667,...,76.333333,1.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,1
2266,Larissa Pacheco,Women's Bantamweight,0.0,0.0,0.0,4.823529,4.200000,15.000000,4.333333,17.576923,...,43.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2352,Tina Lahdemaki,Women's Strawweight,0.0,0.0,0.0,4.823529,4.200000,15.000000,4.333333,17.576923,...,43.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2798,Sheila Gaff,Women's Bantamweight,0.0,0.0,0.0,6.000000,6.000000,1.000000,0.000000,1.000000,...,72.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [18]:
stats_new = combine_male_female_df_new.drop(columns = ["B_fighter", "R_current_lose_streak", "weight_class"])

In [19]:
stats_new

,B_current_win_streak,B_draw,B_avg_BODY_landed,B_avg_CLINCH_landed,B_avg_DISTANCE_landed,B_avg_GROUND_landed,B_avg_HEAD_landed,B_avg_KD,B_avg_LEG_landed,B_avg_PASS,...,B_avg_opp_TOTAL_STR_landed,B_total_title_bouts,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_wins,Gender
0,4.0,0.0,6.000000,0.000000,20.600000,2.000000,11.200000,0.800000,5.400000,0.400000,...,19.200000,0.0,0.0,1.0,0.0,2.0,1.0,0.0,4.0,2
2,3.0,0.0,11.322581,4.387097,38.580645,3.806452,23.258065,0.645161,12.193548,0.935484,...,49.774194,1.0,0.0,0.0,7.0,10.0,6.0,0.0,23.0,2
3,4.0,0.0,14.000000,11.000000,48.750000,10.500000,53.750000,0.500000,2.500000,0.500000,...,34.250000,0.0,0.0,0.0,2.0,2.0,0.0,0.0,4.0,2
4,1.0,0.0,14.500000,2.000000,59.500000,0.000000,45.000000,0.000000,2.000000,0.000000,...,90.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2
6,3.0,0.0,7.416667,3.416667,34.666667,3.416667,24.000000,0.416667,10.083333,0.583333,...,46.500000,0.0,0.0,0.0,2.0,3.0,3.0,0.0,8.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2073,1.0,0.0,4.000000,3.666667,9.333333,6.666667,12.666667,0.000000,3.000000,1.333333,...,76.333333,1.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,1
2266,0.0,0.0,4.823529,4.200000,15.000000,4.333333,17.576923,0.125000,3.583333,1.000000,...,43.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2352,0.0,0.0,4.823529,4.200000,15.000000,4.333333,17.576923,0.125000,3.583333,1.000000,...,43.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2798,0.0,0.0,6.000000,6.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,72.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [22]:
import numpy as np
corrMatrix = newView.corr()
print (corrMatrix)
ax = sns.heatmap(corrMatrix)
ax


                              R_current_lose_streak  B_current_win_streak  \
R_current_lose_streak                      1.000000             -0.203020   
B_current_win_streak                      -0.203020              1.000000   
B_draw                                          NaN                   NaN   
B_avg_BODY_landed                          0.047281              0.173554   
B_avg_CLINCH_landed                        0.024998              0.121377   
B_avg_DISTANCE_landed                      0.030129              0.193924   
B_avg_GROUND_landed                       -0.049876              0.256107   
B_avg_HEAD_landed                         -0.005453              0.243542   
B_avg_KD                                  -0.036641              0.183342   
B_avg_LEG_landed                           0.001944              0.157720   
B_avg_PASS                                -0.092800              0.225700   
B_avg_REV                                  0.010352              0.043868   

NameError: name 'sns' is not defined

In [ ]:
knn = NearestNeighbors(metric = "cosine", algorithm = "brute")
knn.fit(stats_new)

In [67]:
fighter_indices_new = knn.kneighbors(stats_new)[1]
fighter_indices_new
len(fighter_indices_new)
stats_new

,B_current_win_streak,B_draw,B_avg_BODY_landed,B_avg_CLINCH_landed,B_avg_DISTANCE_landed,B_avg_GROUND_landed,B_avg_HEAD_landed,B_avg_KD,B_avg_LEG_landed,B_avg_PASS,...,B_avg_opp_TOTAL_STR_landed,B_total_title_bouts,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_wins,Gender
0,4.0,0.0,6.000000,0.000000,20.600000,2.000000,11.200000,0.800000,5.400000,0.400000,...,19.200000,0.0,0.0,1.0,0.0,2.0,1.0,0.0,4.0,2
2,3.0,0.0,11.322581,4.387097,38.580645,3.806452,23.258065,0.645161,12.193548,0.935484,...,49.774194,1.0,0.0,0.0,7.0,10.0,6.0,0.0,23.0,2
3,4.0,0.0,14.000000,11.000000,48.750000,10.500000,53.750000,0.500000,2.500000,0.500000,...,34.250000,0.0,0.0,0.0,2.0,2.0,0.0,0.0,4.0,2
4,1.0,0.0,14.500000,2.000000,59.500000,0.000000,45.000000,0.000000,2.000000,0.000000,...,90.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2
6,3.0,0.0,7.416667,3.416667,34.666667,3.416667,24.000000,0.416667,10.083333,0.583333,...,46.500000,0.0,0.0,0.0,2.0,3.0,3.0,0.0,8.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2073,1.0,0.0,4.000000,3.666667,9.333333,6.666667,12.666667,0.000000,3.000000,1.333333,...,76.333333,1.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,1
2266,0.0,0.0,4.823529,4.200000,15.000000,4.333333,17.576923,0.125000,3.583333,1.000000,...,43.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2352,0.0,0.0,4.823529,4.200000,15.000000,4.333333,17.576923,0.125000,3.583333,1.000000,...,43.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2798,0.0,0.0,6.000000,6.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,72.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [68]:
def get_recommends(fighter):
    stats_rindex = stats_new.reset_index(drop=True)
    combine_male_female_df_new_rindex = combine_male_female_df_new.reset_index(drop=True)
    dist, fighter_indices = knn.kneighbors(stats_rindex)
    fighters = fighter_indices[combine_male_female_df_new_rindex.loc[combine_male_female_df_new_rindex['B_fighter'] == fighter].index].tolist()[0]
    fighters_as_df = combine_male_female_df_new_rindex[combine_male_female_df_new_rindex.index.isin(fighters)]
    fighters_as_df = fighters_as_df[fighters_as_df['B_fighter'] != fighter]
    return fighters_as_df

In [69]:
get_recommends("Amanda Nunes")

,B_fighter,weight_class,R_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_landed,B_avg_CLINCH_landed,B_avg_DISTANCE_landed,B_avg_GROUND_landed,B_avg_HEAD_landed,...,B_avg_opp_TOTAL_STR_landed,B_total_title_bouts,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_wins,Gender
126,Diego Ferreira,Lightweight,0.0,3.0,0.0,6.428571,2.857143,20.428571,8.285714,22.285714,...,27.857143,0.0,0.0,0.0,1.0,3.0,1.0,0.0,5.0,2
566,Frankie Edgar,Featherweight,1.0,5.0,0.0,12.210526,5.789474,40.368421,16.315789,41.263158,...,50.684211,7.0,0.0,0.0,8.0,4.0,2.0,0.0,14.0,2
690,Fabricio Werdum,Heavyweight,0.0,5.0,0.0,6.666667,5.333333,31.111111,6.444444,28.111111,...,31.000000,1.0,0.0,0.0,2.0,4.0,1.0,0.0,7.0,2
1033,Forrest Griffin,Light Heavyweight,1.0,2.0,0.0,7.769231,8.384615,40.076923,6.000000,31.846154,...,43.384615,3.0,0.0,1.0,5.0,1.0,2.0,0.0,9.0,2


In [70]:
get_recommends("Khabib Nurmagomedov")

,B_fighter,weight_class,R_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_landed,B_avg_CLINCH_landed,B_avg_DISTANCE_landed,B_avg_GROUND_landed,B_avg_HEAD_landed,...,B_avg_opp_TOTAL_STR_landed,B_total_title_bouts,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_wins,Gender
104,Nad Narimani,Featherweight,0.0,2.0,0.0,7.000,7.500,35.50,21.500,50.000,...,54.50,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,2
510,Rustam Khabilov,Lightweight,0.0,3.0,0.0,2.875,2.375,13.50,8.000,17.875,...,27.00,0.0,0.0,0.0,4.0,1.0,0.0,1.0,6.0,2
751,Dennis Bermudez,Featherweight,0.0,7.0,0.0,6.125,11.750,23.75,17.125,38.125,...,48.75,1.0,0.0,2.0,2.0,1.0,2.0,0.0,7.0,2
879,Chris Weidman,Middleweight,0.0,5.0,0.0,2.800,3.800,16.20,8.200,21.800,...,22.80,0.0,0.0,0.0,2.0,1.0,2.0,0.0,5.0,2
